In [1]:
import rasterio
import numpy as np
import geopandas as gpd
import folium
import shapely
from shapely.geometry import Polygon
from rasterio import features
from IPython.display import display

minx = 252500

cover_2019_path = 'data/raster/lidar_products/2019_lidar_cover.tif'
cover_2020_path = 'data/raster/lidar_products/2020_lidar_cover.tif'

def raster_to_polygon_with_minx(raster_path, minx):
    with rasterio.open(raster_path) as src:
        raster_array = src.read(1)
        
        # Adjust extents based on minx
        xmin, ymin, xmax, ymax = src.bounds
        if xmin < minx:
            
            x_offset = int((minx - xmin) / src.res[0])
            raster_array = raster_array[:, x_offset:]
            
            new_transform = rasterio.transform.from_origin(minx, ymax, src.res[0], src.res[1])
        else:
            new_transform = src.transform
        
        mask = raster_array <= 100
        
        # Generate shapes from the mask
        shapes = features.shapes(mask.astype('int16'), mask=mask, transform=new_transform)
        
        # Convert shapes to a GeoDataFrame
        polygons = []
        for geom, value in shapes:
            if value == 1:
                polygons.append(Polygon(geom['coordinates'][0]))
        
        gdf = gpd.GeoDataFrame({'geometry': polygons}, crs=src.crs)
        return gdf


gdf_2019 = raster_to_polygon_with_minx(cover_2019_path, minx)
gdf_2020 = raster_to_polygon_with_minx(cover_2020_path, minx)

gdf_2019_4326 = gdf_2019.to_crs(epsg=4326)
gdf_2020_4326 = gdf_2020.to_crs(epsg=4326)

# Create a folium map
combined_gdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries(list(gdf_2019_4326.geometry) + list(gdf_2020_4326.geometry)))
centroid = combined_gdf.geometry.unary_union.centroid
m = folium.Map(location=[centroid.y, centroid.x], zoom_start=12, tiles="OpenStreetMap")

tiles = folium.TileLayer('https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite')
tiles.add_to(m)

# Add GeoDataFrames to the map
folium.GeoJson(gdf_2019_4326, style_function=lambda feature: {
    'fillColor': 'green',
    'color': 'green',
    'weight': 1,
    'fillOpacity': 0.3
}).add_to(m)

folium.GeoJson(gdf_2020_4326, style_function=lambda feature: {
    'fillColor': 'blue',
    'color': 'blue',
    'weight': 1,
    'fillOpacity': 0.3
}).add_to(m)

display(m)

/var/folders/vp/1c12nhtx09ng562vszvlczthvwd07y/T/ipykernel_22032/1507191417.py:53: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  centroid = combined_gdf.geometry.unary_union.centroid


In [2]:
intersection = gpd.overlay(gdf_2019, gdf_2020, how='intersection')
gdf_2019_only = gpd.overlay(gdf_2019, gdf_2020, how='difference')

intersection_4326 = intersection.to_crs(epsg=4326)
gdf_2019_only_4326 = gdf_2019_only.to_crs(epsg=4326)

combined_gdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries(list(intersection_4326.geometry) + list(gdf_2019_only_4326.geometry)))
centroid = combined_gdf.geometry.unary_union.centroid
m = folium.Map(location=[centroid.y, centroid.x], zoom_start=12, tiles="OpenStreetMap")

tiles = folium.TileLayer('https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite')
tiles.add_to(m)


folium.GeoJson(intersection_4326, style_function=lambda feature: {
    'fillColor': 'purple',
    'color': 'purple',
    'weight': 1,
    'fillOpacity': 0.3
}).add_to(m)

folium.GeoJson(gdf_2019_only_4326, style_function=lambda feature: {
    'fillColor': 'green',
    'color': 'green',
    'weight': 1,
    'fillOpacity': 0.3
}).add_to(m)

display(m)


/var/folders/vp/1c12nhtx09ng562vszvlczthvwd07y/T/ipykernel_22032/110592257.py:8: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  centroid = combined_gdf.geometry.unary_union.centroid


In [3]:
test_path = 'data/vector/test_areas.geojson'
train_path = 'data/vector/train_areas.geojson'

intersection.to_file(test_path, driver='GeoJSON')
gdf_2019_only.to_file(train_path, driver='GeoJSON')

In [4]:
path_to_rap_template = 'data/raster/rap/woody_cover_1986.tif'
import rasterio
import geopandas as gpd
from shapely.geometry import Polygon

with rasterio.open(path_to_rap_template) as src:
    
    # Get the raster's bounding box in its native CRS
    left, bottom, right, top = src.bounds
    
    # Get the raster's transform
    transform = src.transform
    
    # Get the raster's width and height
    width = src.width
    height = src.height
    
    # Get the spatial resolution
    resolution_x, resolution_y = src.res
    
    # Create a list to hold the cell centroids
    cell_centroids = []
    
    # Loop through each cell in the raster
    for row in range(height):
        for col in range(width):
            # Calculate the coordinates of the cell's corners
            x1, y1 = transform * (col, row)
            x2, y2 = transform * (col + 1, row)
            x3, y3 = transform * (col + 1, row + 1)
            x4, y4 = transform * (col, row + 1)
            
            # Create a polygon from the cell's corners
            polygon = Polygon([(x1, y1), (x2, y2), (x3, y3), (x4, y4)])
            
            # Get the centroid of the polygon
            centroid = polygon.centroid
            
            # Add the centroid to the list
            cell_centroids.append(centroid)
            
    # Create a GeoDataFrame from the cell centroids
    rap_grid_centroids = gpd.GeoDataFrame(geometry=cell_centroids, crs=src.crs)


In [9]:
def subset_points(points_gdf, area_gdf, area_name):
    """Subsets points within a given area and saves to a geojson.

    Args:
        points_gdf (geopandas.GeoDataFrame): GeoDataFrame of points.
        area_gdf (geopandas.GeoDataFrame): GeoDataFrame of area polygon.
        area_name (str): Name of the area (for saving).
    """
    points_in_area = gpd.sjoin(points_gdf, area_gdf, predicate='within')
    print(f'{area_name} points: {len(points_in_area)}')
    points_in_area = points_in_area[['geometry']]
    points_in_area.to_file(f'data/vector/{area_name}_points.geojson', driver='GeoJSON')
    return points_in_area

train_area = gpd.read_file('data/vector/train_areas.geojson')
test_area = gpd.read_file('data/vector/test_areas.geojson')

train_points = subset_points(rap_grid_centroids, train_area, 'train')
test_points = subset_points(rap_grid_centroids, test_area, 'test')


train points: 33879
test points: 2893
